In [37]:
import pandas as pd
import numpy as np
import datetime
from sklearn.linear_model import LinearRegression
import pandas_datareader as pdr
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [38]:
regressor=LinearRegression()
dic=pd.DataFrame()
suppliers=pd.read_csv('supplier_info.csv')
i=0
df=pd.DataFrame()
market=pdr.get_data_yahoo('^gspc',start=datetime.datetime(2013,2,27),end=datetime.datetime(2014,1,7))
market_return=market['Adj Close'].pct_change()
for index,row in suppliers.iterrows():
    t_name=row['ticker'].strip()
    try:
        name = pdr.get_data_yahoo(t_name, 
                          start=datetime.datetime(2013,2,27), 
                          end=datetime.datetime(2014,1,7))
        daily_returns = name['Adj Close'].pct_change()
        daily_returns.columns=['Actual']
        regressor.fit(np.array(market_return['2013-02-28':'2013-12-11']).reshape(-1,1), list(daily_returns['2013-02-28':'2013-12-11']))
        dic.loc[:,'AR']=daily_returns['2013-02-28':'2014-01-07']-regressor.predict(np.array(market_return['2013-02-28':'2014-01-07']).reshape(-1,1))
        df.loc[i,'tic']=t_name
        df.loc[i,'AR-1']=dic.loc['2013-12-20','AR']
        df.loc[i,'CIO']=row['CIO']
        df.loc[i,'CMO']=row['CMO']
        df.loc[i,'retail']=row['Retail']
        df.loc[i,'%Revenue']=row['%Revenue']
        df.loc[i,'gvkey']=row['gvkey']
        i+=1
    except:continue

In [39]:
event_study=df[df['retail'].isin([0])]
event_study=event_study[(event_study['tic']!='APTV') & (event_study['tic']!='NKE') & (event_study['tic']!='UFI')
                        & (event_study['tic']!='VFC') & (event_study['tic']!='TR') & (event_study['tic']!='QUAD')
                      & (event_study['tic']!='HELE') & (event_study['tic']!='GAIA') & (event_study['tic']!='JAKK')
                       & (event_study['tic']!='REV') & (event_study['tic']!='JBSS') & (event_study['tic']!='SMDM')]
event_study['%Revenue']=event_study['%Revenue'].str.strip("%").astype(float)/100
event_study=event_study.sort_values('%Revenue')
event_study=event_study.dropna(subset=['%Revenue'])
little=event_study[0:31] 
medium=event_study[31:70] 
large=event_study[70:] 

print('Suppliers with low dependence:')
print(little['AR-1'].mean())
print(stats.ttest_1samp(little['AR-1'],0.0))
print('Suppliers with medium dependence:')
print(medium['AR-1'].mean())
print(stats.ttest_1samp(medium['AR-1'],0.0))
print('Suppliers with high dependence:')
print(large['AR-1'].mean())
print(stats.ttest_1samp(large['AR-1'],0.0))

name_list = ['Low','Medium','High']
num_list = [little['AR-1'].mean(),medium['AR-1'].mean(),large['AR-1'].mean()]

plt.xlabel("Percentage of revenue coming from Target")  
plt.ylabel("Abnormal return") 
plt.bar(range(len(num_list)), num_list,color='rgb',tick_label=name_list)
plt.title('Average Abnormal Return on Day 1')
plt.show()


Suppliers with low dependence:
0.008933743823658339
Ttest_1sampResult(statistic=2.6450091683547274, pvalue=0.012874010794363722)
Suppliers with medium dependence:
0.0021327843951965933
Ttest_1sampResult(statistic=0.4765075187708817, pvalue=0.636441534957136)
Suppliers with high dependence:
-0.0034230056853702096
Ttest_1sampResult(statistic=-1.9370302470619705, pvalue=0.06161234938882388)


In [40]:
info=pd.read_csv('info2012.csv')
all_info= pd.merge(info, df, how='left', on='tic')

In [41]:
# This takes a lot of time
tgt=pd.read_csv('tgt_holding.csv')
s=pd.read_csv('supplier_holding.csv')
overlap=pd.DataFrame()
i=0
for index,row in s.iterrows():
    if s.loc[i,'mgrname'] in list(tgt['mgrname']):
        overlap.loc[i,'mgrname']=s.loc[i,'mgrname']
        overlap.loc[i,'share']=s.loc[i,'shares']
        overlap.loc[i,'company']=s.loc[i,'stkname']
    i+=1
dictionary={}
for index,row in s.iterrows():
    if row['stkname'] in dictionary:
        dictionary[row['stkname']]+=row['shares']
    else:
        dictionary[row['stkname']]=row['shares']


In [42]:
shares_holding=overlap.groupby('company').share.sum()
shares_holding = shares_holding.reset_index()

for index,row in shares_holding.iterrows():
    shares_holding.loc[index,'share_total']=dictionary[shares_holding.loc[index,'company']]
s=s.rename(columns={'stkname':'company'})
s=s.rename(columns={'ticker':'tic'})
s=s.drop_duplicates(subset = 'company', keep = 'first')
investor_overlap=pd.merge(shares_holding,s,on='company',how='right')
investor_overlap['invester_overlap']=shares_holding['share']/shares_holding['share_total']
all_info2=pd.merge(all_info,investor_overlap,on='tic',how='left')

In [43]:
variables=pd.DataFrame()
variables['ticker']=all_info['tic']
variables['AAR']=all_info['AR-1']
variables['firm_size']=np.log(all_info['at'])
variables['COGS/Sale']=all_info['cogs']/all_info['sale']
variables['GP/Sale']=(all_info['revt']-all_info['cogs'])/all_info['sale']
variables['SG&A/Sale']=all_info['xsga']/all_info['sale']
variables['RoA']=all_info['ni']/all_info['at']
variables['RoE']=all_info['ni']/(all_info['csho']*all_info['prcc_f'])
variables['RoI']=all_info['ni']/all_info['icapt']
variables['RoS']=all_info['ni']/all_info['revt']
variables['CIO']=all_info['CIO']
variables['CMO']=all_info['CMO']
variables['cash_holding']=all_info['che']/all_info['at']
variables['inventory_turnover']=all_info['cogs']/((all_info['invt']+all_info['invt'])/2)
variables['invester_overlap']=all_info2['invester_overlap']
variables['%revenue']=all_info['%Revenue'].str.strip("%").astype(float)/100
variables['%revenue']=variables['%revenue']
variables=variables.replace([np.inf, -np.inf], np.nan, inplace=False)
variables=variables.dropna(axis=0,how='any')
variables.corr()

AAR  firm_size  COGS/Sale   GP/Sale  SG&A/Sale  \
AAR                 1.000000  -0.144656   0.112884 -0.112884  -0.115676   
firm_size          -0.144656   1.000000  -0.363242  0.363242   0.045117   
COGS/Sale           0.112884  -0.363242   1.000000 -1.000000  -0.847686   
GP/Sale            -0.112884   0.363242  -1.000000  1.000000   0.847686   
SG&A/Sale          -0.115676   0.045117  -0.847686  0.847686   1.000000   
RoA                 0.099074   0.302559  -0.236291  0.236291  -0.062077   
RoE                 0.169511   0.240360  -0.098323  0.098323  -0.086297   
RoI                 0.069443   0.320442  -0.207428  0.207428  -0.045241   
RoS                 0.011694   0.455572  -0.479395  0.479395   0.091360   
CIO                 0.317306   0.267752  -0.053330  0.053330  -0.001668   
CMO                 0.138745   0.287073  -0.176515  0.176515   0.100132   
cash_holding        0.118434  -0.131501  -0.122036  0.122036   0.039549   
inventory_turnover -0.010526  -0.084705   0.079900 -0.079900  -0.172015   
invester_overlap   -0.089166  -0.072202   0.082598 -0.082598  -0.080531   
%revenue            0.018586  -0.334641   0.084295 -0.084295   0.043579   

                         RoA       RoE       RoI       RoS       CIO  \
AAR                 0.099074  0.169511  0.069443  0.011694  0.317306   
firm_size           0.302559  0.240360  0.320442  0.455572  0.267752   
COGS/Sale          -0.236291 -0.098323 -0.207428 -0.479395 -0.053330   
GP/Sale             0.236291  0.098323  0.207428  0.479395  0.053330   
SG&A/Sale          -0.062077 -0.086297 -0.045241  0.091360 -0.001668   
RoA                 1.000000  0.868714  0.919385  0.891585  0.149094   
RoE                 0.868714  1.000000  0.843343  0.727846  0.096600   
RoI                 0.919385  0.843343  1.000000  0.799632  0.158941   
RoS                 0.891585  0.727846  0.799632  1.000000  0.137230   
CIO                 0.149094  0.096600  0.158941  0.137230  1.000000   
CMO                 0.129132  0.007085  0.189623  0.159956  0.536586   
cash_holding        0.260500  0.140844  0.193117  0.259174  0.047665   
inventory_turnover  0.089986  0.075463  0.073516  0.068694 -0.048297   
invester_overlap   -0.028381 -0.015638 -0.079917 -0.059923 -0.398340   
%revenue           -0.158802 -0.110976 -0.114029 -0.208722 -0.044032   

                         CMO  cash_holding  inventory_turnover  \
AAR                 0.138745      0.118434           -0.010526   
firm_size           0.287073     -0.131501           -0.084705   
COGS/Sale          -0.176515     -0.122036            0.079900   
GP/Sale             0.176515      0.122036           -0.079900   
SG&A/Sale           0.100132      0.039549           -0.172015   
RoA                 0.129132      0.260500            0.089986   
RoE                 0.007085      0.140844            0.075463   
RoI                 0.189623      0.193117            0.073516   
RoS                 0.159956      0.259174            0.068694   
CIO                 0.536586      0.047665           -0.048297   
CMO                 1.000000     -0.049502           -0.013527   
cash_holding       -0.049502      1.000000            0.459611   
inventory_turnover -0.013527      0.459611            1.000000   
invester_overlap   -0.169952      0.000817            0.021810   
%revenue           -0.014660      0.333968            0.159723   

                    invester_overlap  %revenue  
AAR                        -0.089166  0.018586  
firm_size                  -0.072202 -0.334641  
COGS/Sale                   0.082598  0.084295  
GP/Sale                    -0.082598 -0.084295  
SG&A/Sale                  -0.080531  0.043579  
RoA                        -0.028381 -0.158802  
RoE                        -0.015638 -0.110976  
RoI                        -0.079917 -0.114029  
RoS                        -0.059923 -0.208722  
CIO                        -0.398340 -0.044032  
CMO                        -0.169952 -0.014660  
cash_holding              

In [44]:
X=variables.iloc[:,2:]
y=variables.loc[:,'AAR']
X2 = sm.add_constant(X)
est = sm.OLS(y,X2).fit()
print(est.summary())

                            OLS Regression Results                            
Dep. Variable:                    AAR   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     2.068
Date:                Wed, 06 May 2020   Prob (F-statistic):             0.0278
Time:                        18:42:39   Log-Likelihood:                 230.56
No. Observations:                  81   AIC:                            -433.1
Df Residuals:                      67   BIC:                            -399.6
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0434      0